In [1]:
%matplotlib inline

import intake
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from explore_utils import get_cmip6_catalogue
from extract_utils import find_overlap_models, rename_dimensions

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['geojson', 'postgis', 'shapefile', 'spatialite'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
# Select which datasets are required by populating these lists with details
dset_name, exp, var, table_id, grid_label = [], [], [], [], []

# Select name for this clollection of inputs
collection_name = 'historical'

# 1. siconc
dset_name.append('siconc')
exp.append('historical')
var.append('siconc')
table_id.append('SImon')
grid_label.append('gn')

# 2. areacello
dset_name.append('areacello')
exp.append('historical')
var.append('areacello')
table_id.append('Ofx')
grid_label.append('gn')

# 3. tas
dset_name.append('tas')
exp.append('historical')
var.append('tas')
table_id.append('Amon')
grid_label.append('gn')

In [2]:
# Select which datasets are required by populating these lists with details
dset_name, exp, var, table_id, grid_label = [], [], [], [], []

# Select name for this clollection of inputs
collection_name = 'piControl'

# 1. siconc
dset_name.append('siconc')
exp.append('piControl')
var.append('siconc')
table_id.append('SImon')
grid_label.append('gn')

# 2. areacello
dset_name.append('areacello')
exp.append('piControl')
var.append('areacello')
table_id.append('Ofx')
grid_label.append('gn')

# 3. tas
dset_name.append('tas')
exp.append('piControl')
var.append('tas')
table_id.append('Amon')
grid_label.append('gn')

In [3]:
# Get full catalogue of CMIP6 data on glade or cloud
cmip6_collection = get_cmip6_catalogue()

In [4]:
# Find where models contain all necessary variables
models_intersect = find_overlap_models(dset_name, exp, var, table_id, grid_label, cmip6_collection)
print(models_intersect)

['MIROC6', 'ACCESS-ESM1-5', 'SAM0-UNICON', 'NorESM1-F', 'HadGEM3-GC31-LL', 'CESM2-FV2', 'CESM2', 'ACCESS-CM2', 'CESM2-WACCM-FV2', 'MRI-ESM2-0', 'CESM2-WACCM', 'CanESM5', 'MIROC-ES2L', 'MPI-ESM1-2-HR']


In [5]:
# Get dictionary of file names for speficied data
dset_dict = {}
for i in range(0, len(dset_name)):
    dset_dict[dset_name[i]] = cmip6_collection.search(
                                experiment_id=exp[i], table_id=table_id[i], 
                                variable_id=var[i], grid_label=grid_label[i])

In [13]:
for d in dset_dict.keys():
    print(d)
    print(dset_dict[d].)

siconc
pangeo-cmip6-ESM Collection with 31 entries:
	> 1 activity_id(s)

	> 18 institution_id(s)

	> 27 source_id(s)

	> 1 experiment_id(s)

	> 6 member_id(s)

	> 1 table_id(s)

	> 1 variable_id(s)

	> 1 grid_label(s)

	> 31 zstore(s)

	> 0 dcpp_init_year(s)

areacello
pangeo-cmip6-ESM Collection with 32 entries:
	> 1 activity_id(s)

	> 17 institution_id(s)

	> 28 source_id(s)

	> 1 experiment_id(s)

	> 6 member_id(s)

	> 1 table_id(s)

	> 1 variable_id(s)

	> 1 grid_label(s)

	> 32 zstore(s)

	> 0 dcpp_init_year(s)

tas
pangeo-cmip6-ESM Collection with 36 entries:
	> 1 activity_id(s)

	> 18 institution_id(s)

	> 28 source_id(s)

	> 1 experiment_id(s)

	> 9 member_id(s)

	> 1 table_id(s)

	> 1 variable_id(s)

	> 1 grid_label(s)

	> 36 zstore(s)

	> 0 dcpp_init_year(s)



In [6]:
# Loading data
for d in dset_dict.keys():
    print(d)
    dset_dict[d] = dset_dict[d].to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True}, 
                                           cdf_kwargs={'chunks': {}, 'decode_times': True})
    
# Having some problems, times don't want to be decoded for picontrol. Might not be a problem but should investigate.


siconc
Progress: |-------------------------------------------------------------------------------| 0.0% 

/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.7/site

Progress: |████████████-------------------------------------------------------------------| 14.8% 

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Progress: |█████████████████████████████████████████████████████--------------------------| 66.7% 

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


RecursionError: maximum recursion depth exceeded while calling a Python object

In [7]:
# Rename dimensions to i,j so they're consistent across variables
dset_dict_temp = {}
for d in dset_dict.keys():
    dset_dict_temp[d] = {}
    for m in dset_dict[d].keys():
        dset_dict_temp[d][m] = rename_dimensions(dset_dict[d][m], dset_dict_temp)

dset_dict = dset_dict_temp

In [8]:
# Making key of dataset model name
dset_dict_temp = {}
for d in dset_dict.keys():
    dset_dict_temp[d] = {}
    for key, item in dset_dict[d].items():
        model = item.attrs['source_id']
        if model in models_intersect:
            dset_dict_temp[d][model] = item

dset_dict = dset_dict_temp

In [9]:
# Adding areacello and/or areacella as a variable in other datasets
if 'areacello' in dset_dict.keys():
    for d in dset_dict.keys(): # for each variable
        for key in dset_dict[d].keys(): # for each model
            # if table_id suggests variable is a sea ice or ocean variable, add areacello
            if dset_dict[d][key].attrs['table_id'][0] in ['S', 'O']:
                dset_dict[d][key]['areacello'] = dset_dict['areacello'][key]['areacello']

if 'areacella' in dset_dict.keys():
    for d in dset_dict.keys(): # for each variable
        for key in dset_dict[d].keys(): # for each model
            # if table_id suggests variable is an atmosphere variable, add areacella
            if dset_dict[d][key].attrs['table_id'][0] in ['A']:
                dset_dict[d][key]['areacella'] = dset_dict['areacella'][key]['areacella']                

In [10]:
# Ensure only ensemble members that overlap all data sets are included
dset_dict_temp = {}

for d in dset_dict.keys():
    dset_dict_temp[d] = {}
    
for m in models_intersect:
    ems = [0]
    for d in dset_dict.keys():
        dset_dict_temp[d][m] = {}
        if d is not 'areacello':
            if ems[0]==0:
                ems = dset_dict[d][m]['member_id'].values
            else:
                ems = list(set(ems) & set(dset_dict[d][m]['member_id'].values))
    
    for d in dset_dict.keys():                       
        dset_dict_temp[d][m] = dset_dict[d][m].sel(member_id=ems)
        
    print(m, ems)
    print()
        
dset_dict = dset_dict_temp

NorCPM1 ['r26i1p1f1', 'r17i1p1f1', 'r23i1p1f1']

MIROC-ES2L ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2']

NorESM2-LM ['r2i1p1f1']

MPI-ESM1-2-HR ['r4i1p1f1', 'r2i1p1f1', 'r9i1p1f1']

CanESM5-CanOE ['r1i1p2f1', 'r3i1p2f1', 'r2i1p2f1']

CESM2 ['r4i1p1f1', 'r5i1p1f1', 'r10i1p1f1', 'r7i1p1f1', 'r1i1p1f1', 'r8i1p1f1', 'r2i1p1f1', 'r11i1p1f1', 'r6i1p1f1', 'r3i1p1f1', 'r9i1p1f1']

MRI-ESM2-0 ['r4i1p1f1', 'r5i1p1f1', 'r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1']

CanESM5 ['r5i1p1f1', 'r24i1p1f1', 'r3i1p2f1', 'r17i1p1f1', 'r1i1p1f1', 'r2i1p2f1', 'r17i1p2f1', 'r1i1p2f1', 'r25i1p1f1', 'r4i1p2f1', 'r3i1p1f1', 'r13i1p1f1', 'r12i1p1f1', 'r5i1p2f1', 'r13i1p2f1', 'r8i1p1f1', 'r2i1p1f1', 'r11i1p1f1', 'r6i1p1f1', 'r16i1p1f1', 'r21i1p1f1', 'r6i1p2f1', 'r25i1p2f1', 'r4i1p1f1', 'r19i1p2f1', 'r7i1p2f1', 'r18i1p2f1', 'r7i1p1f1', 'r19i1p1f1', 'r22i1p2f1', 'r9i1p2f1', 'r20i1p2f1', 'r9i1p1f1', 'r21i1p2f1', 'r20i1p1f1', 'r18i1p1f1', 'r8i1p2f1', 'r14i1p1f1', 'r10i1p1f1', 'r15i1p2f1', 'r24i1p2f1', 'r12i1p2f1', 'r14i1p2f1', 'r22i1p1

In [11]:
# Save dictionaries for future use
save_flag = True
if save_flag:
    if dset_dict:
        np.save('dset_dict_temp_' + collection_name + '.npy', dset_dict)

In [ ]:
# Problems
#1. Updated pi-control model list. Now intake-cesm not working...

In [12]:
# To-do
#1. add something for selecting specific time interval